In [5]:
import os
print("當前工作目錄:", os.getcwd())

當前工作目錄: c:\Users\User\OneDrive\桌面Dell\富邦\FBI_Intern


In [6]:
import json
import requests
import time
import re
import os
import warnings
import pandas as pd
from requests.packages.urllib3.exceptions import InsecureRequestWarning
from bs4 import BeautifulSoup
import urllib
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
""" 初始化 """
# 忽略不安全連線警告
warnings.simplefilter('ignore', InsecureRequestWarning)
user_agent = UserAgent()

# 使用header偽裝成edge瀏覽器下瀏覽行為
headers = {'content-type': 'text/html; charset = UTF-8',
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7",
    "Referer": "https://www.learncodewithmike.com/",
    "Sec-Ch-Ua": "\"Not/A)Brand\";v=\"99\", \"Google Chrome\";v=\"115\", \"Chromium\";v=\"115\"",
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": "\"Windows\"",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "cross-site",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": user_agent.random,
    "X-Amzn-Trace-Id": "Root=1-64d18fe1-1f5ba9715c698e6340cfa803"
          }

# 假設這是處理單個商戶的函數


sleep_time = 10

def timer(func):
    def wrapper(*args, **kwargs):
        tic = time.time()
        value = func(*args, **kwargs)
        print('Time consumed: {} sec.'.format(time.time()-tic))
        return value
    return wrapper



class Merchant:
    def __init__(self, name, keyword, page):
        self.name = name
        self.page = page
        self.keyword = keyword
        self.get_html()
        self.parse_attr()

    def get_html(self):
        url = 'https://google.com/search?q="{}"+{}&start={}'.format(urllib.parse.quote(self.name), self.keyword, (self.page-1)*10)
        # 創建 WebDriver 實例
        service = Service('chromedriver.exe')
        # 添加無頭模式選項
        options = webdriver.ChromeOptions()
        #options.add_argument('--headless')  
        driver = webdriver.Chrome(service=service, options=options)
        
        driver.get(url)
        count = 0

        # 獲取 HTML 源代碼
        self.html = driver.page_source

        # 關閉 WebDriver 實例
        driver.quit()
        while count < 3:
            try:
                result = requests.get(url, verify=False, auth=('user', 'pass'), headers=headers)
                if result.status_code == 200:
                    self.html = result.text
                    break
                elif result.status_code == 429:
                    count = count+1
                    time.sleep(60)
            except:
                time.sleep(sleep_time)
                count = count+1
        else:
            raise ConnectionError('Cannot get the html')

    def parse_attr(self):
        soup = BeautifulSoup(self.html, 'html.parser')
        self.soup = soup

        self.top_address = [element.text for element in soup.findAll('div', {'class':'sXLaOe'})]

        self.title_list = [element.find('h3').text for element in soup.findAll('div', {'class':'yuRUbf'})]
        self.summary_list = [element.text for element in soup.findAll('div', {'class':'VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf'})]


    def to_dict(self, drop_html=True):

        result = {
                    'name' : self.name,
                    'top_address' : self.top_address,
                    'title_list' : self.title_list,
                    'summary_list' : self.summary_list,
                 }
        if not drop_html:
            result['html'] = self.html
        return result


In [7]:
tag = pd.read_excel('特店名&標籤.xlsx')
brands = []
tag_list = []
for i in range(len(tag)):
  brands.append([])
  brands[i].append(tag['name'][i])
  brands[i].append(tag['label'][i].split(","))
for element in brands:
  for j in element[1]:
    tag_list.append(j)
tag_list = set(tag_list)
pages = 3
keywords = ['品牌']
brands = brands[:3]
brands

[['Netflix', ['科技嘗鮮族', '影音串流']],
 ['Spotify', ['科技嘗鮮族', '影音串流']],
 ['KKBOX', ['科技嘗鮮族', '影音串流']]]

In [8]:
# Function to handle individual merchant
def process_merchant(brand, keyword, page):
    try:
        return Merchant(brand, keyword, page).to_dict()
    except Exception as e:
        print(f'Error with {brand} page {page}: {e}')
        return None

# ... Initialization and other code ...

fp_output = r'C:/Users/User/OneDrive/桌面Dell/富邦/result_{}_{}.json'
fp_job_status = r'C:/Users/User/OneDrive/桌面Dell/富邦/job_status_{}_{}.json'

sleep_time = 5

start_time = time.time()

for i, keyword in enumerate(keywords):
    if i != 0:
        time.sleep(30)
    job_status = {'fail_list': [], 'success_list': [], 'todo': [sublist[0] for sublist in brands], 'done_count': 0}
    c = 0
    for brand in job_status['todo']:
        if c >= 45 and c % 5 == 0:
            time.sleep(30)
        c += 1
        result = list()
        print(c, brand)

        # Using a thread pool to process merchants in parallel
        #這裡使用 ThreadPoolExecutor 來並行處理每個品牌的不同頁面。這應該會顯著減少爬取整個網站所需的時間。
        with ThreadPoolExecutor() as executor:
            future_results = [executor.submit(process_merchant, brand, keyword, page) for page in range(1, pages + 1)]
            for future in as_completed(future_results):
                res = future.result()
                if res is not None:
                    result.append(res)

        # ... Rest of the code ...

        with open(fp_output.format(keyword, pages), 'w') as f:
            json.dump(result, f)
        with open(fp_job_status.format(keyword, pages), 'w') as f:
            json.dump(job_status, f)

        print('Keyword: {}, page: {} done.'.format(keyword, pages))
        print('===========================================================================')


1 Netflix


In [ ]:
# Outside the loop, define a list to store all results
all_results = []

start_time = time.time()

for i, keyword in enumerate(keywords):
    # ... rest of the code ...
    
    for brand in job_status['todo']:
        # ... rest of the code ...
        
        result = list()
        print(c, brand)

        with ThreadPoolExecutor() as executor:
            future_results = [executor.submit(process_merchant, brand, keyword, page) for page in range(1, pages + 1)]
            for future in as_completed(future_results):
                res = future.result()
                if res is not None:
                    result.append(res)

        # Append the result for this brand to the all_results list
        all_results.append(result)

        # ... Rest of the code ...

# After the loop, write all results to the file
with open(fp_output.format(keyword, pages), 'w') as f:
    json.dump(all_results, f)


In [11]:
import re


#把爬下來的資料變成df
def create_summary_df(keyword_list:list, brands:list, pages:list):
    """
    input: keyword_list
           brand : 所有品牌的list(多維)，[0]為特店名，[1]為那個特店的label(list)
           pages : 搜尋頁數
    output:組合而成df
    """
    summary_df = pd.DataFrame()
    for keyword in keyword_list:
        for brand_index in range(len(brands)):
            #for page in range(pages):
                with open('C:/Users/User/OneDrive/桌面Dell/富邦/result_品牌_3.json') as f:
                    #with open(f'result_{keyword}_3.json') as f:
                    data = json.load(f)
                for i in data[brand_index]["summary_list"]:
                    #去掉標點符號、日期
                    i = re.sub(r'\W', "", i)
                    i = re.sub(r'\d+年\d+月\d+日', "", i)
                tmp_df = pd.DataFrame({"summary":data[brand_index]["summary_list"]})
                tmp_df['name'] = [data[brand_index]["name"]] * len(tmp_df)
                tmp_df["category"] = [brands[brand_index][1]] * len(tmp_df)
                tmp_df["keyword"] = [keyword] * len(tmp_df)
                summary_df = pd.concat([summary_df,tmp_df])
    return summary_df
summary_df = create_summary_df(keywords, brands, pages)

#看這個特店的標籤是否有在所有label中，建立一個新cloumn
for tag in tag_list:
    summary_df[tag] = summary_df.apply(lambda row: tag in row['category'], axis=1)
summary_df = summary_df.reset_index(drop=True)
summary_df

IndexError: list index out of range

In [ ]:
# fp_output = r'C:/Users/User/OneDrive/桌面Dell/富邦/result_{}_{}.json'
# fp_job_status = r'C:/Users/User/OneDrive/桌面Dell/富邦/job_status_{}_{}.json'


# sleep_time = 5 #每一頁要隔五秒

# start_time = time.time()
# tic = time.time()
# toc = time.time()

# for i, keyword in enumerate(keywords):
#     if i != 0:
#         time.sleep(30) #大量的話要久一點
#     job_status = {'fail_list' : [], 'success_list' : [], 'todo': [sublist[0] for sublist in brands], 'done_count': 0}
#     c = 0
#     for brand in job_status['todo']:
#         if c >= 45 and c%5 == 0:
#           time.sleep(30)
#         c += 1
#         result = list()
#         print(c, brand)
#         for page in range(1,pages+1,1):
#             if time.time()-tic < sleep_time:
#                 time.sleep(sleep_time)

#             tic = time.time()
#             try:
#                 result.append(Merchant(brand, keyword, page).to_dict())
#             except:
#                 print('Error: {} page {} failed.'.format(brand, page))
#                 continue


#     with open(fp_output.format(keyword, page), 'w') as f:
#             f.write(json.dumps(result))
#     with open(fp_job_status.format(keyword, page), 'w') as f:
#         json.dump(job_status, f)

#     print('Keyword: {}, page: {} done.'.format(keyword, page))
#     print('===========================================================================')


1 Netflix
Error: Netflix page 1 failed.


Exception ignored in: <function Service.__del__ at 0x0000019EA0B92790>
Traceback (most recent call last):
  File "c:\Users\User\anaconda3\envs\ml\lib\site-packages\selenium\webdriver\common\service.py", line 193, in __del__
    self.stop()
  File "c:\Users\User\anaconda3\envs\ml\lib\site-packages\selenium\webdriver\common\service.py", line 154, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\User\anaconda3\envs\ml\lib\site-packages\selenium\webdriver\common\service.py", line 138, in send_remote_shutdown_command
    if not self.is_connectable():
  File "c:\Users\User\anaconda3\envs\ml\lib\site-packages\selenium\webdriver\common\service.py", line 127, in is_connectable
    return utils.is_connectable(self.port)
  File "c:\Users\User\anaconda3\envs\ml\lib\site-packages\selenium\webdriver\common\utils.py", line 102, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
  File "c:\Users\User\anaconda3\envs\ml\lib\socket.py", line 832, in create_connectio

Error: Netflix page 2 failed.


: 

In [ ]:
summary_df.to_csv('summary_df.csv')

In [14]:
with open('C:/Users/User/OneDrive/桌面Dell/富邦/job_status_品牌_3.json') as f:
                    #with open(f'result_{keyword}_3.json') as f:
                    data = json.load(f)

In [15]:
data

{'fail_list': [],
 'success_list': [],
 'todo': ['Netflix',
  'Spotify',
  'KKBOX',
  'Disney+',
  '王道銀行',
  'Line Bank',
  '街口支付',
  'GameFi',
  'gogoro',
  'Fitbit'],
 'done_count': 0}